<a href="https://colab.research.google.com/github/hongsy0113/cose461-NLP-project-choi-hong/blob/main/sns_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import json
from pandas import json_normalize


In [6]:
# json data 파일들을 google drive에 저장
# google drive 에 있는 파일들을 접근하기 위해 mount
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
# load json file as below
# 참고로 행사3.json 파일은 디버그 용으로 dialogue 두개만 있는 파일임.
with open('/content/gdrive/MyDrive/cose461/행사3.json', 'r') as f:

    jdata = json.load(f)

In [4]:
# Conver json file to dataFrame
# input file parameter should be str, path and name of json file
# output dataFrame contains utterances. 
# each utterance has dialogueInfo and participantsInfo (i.e gender)

def json2df (file):
  with open(file, 'r') as f:
    jdata = json.load(f)
  
  meta_list = [['header', 'dialogueInfo', 'numberOfParticipants'],
             ['header', 'dialogueInfo', 'numberOfUtterances'],
             ['header', 'dialogueInfo', 'numberOfTurns'],
             ['header', 'dialogueInfo', 'type'],
             ['header', 'dialogueInfo', 'topic'],
             ['header', 'dialogueInfo', 'dialogueID'],
            ]
  # dataframe of utterances 
  df1 = pd.json_normalize(jdata['data'][:], record_path=['body'], meta= meta_list, errors='ignore')
  # dataframe of participants info
  df2 = pd.json_normalize(jdata['data'], record_path=['header', 'participantsInfo'],  meta= [['header', 'dialogueInfo', 'dialogueID']], errors='ignore')
  # join df1 and df2 where (df1.dialogueID == df2.dialogueID and df1.participantID == df2.participantID)
  df = pd.merge(left = df1 , right = df2, how = "left", on = ["header.dialogueInfo.dialogueID", 'participantID' ],sort=False)

  # rename column names
  # 열이름 너무 길어서 바꿈. P는 참가자, U는 메시지, D는 대화, T는 turn 으로 통일시킴. 맘에 안 들면 바꿔도 ok
  df.rename(columns = {'header.dialogueInfo.numberOfParticipants' : 'P_num', 
                       'header.dialogueInfo.numberOfUtterances': 'U_num', 
                       'header.dialogueInfo.numberOfTurns':'T_num',
                       'header.dialogueInfo.type': 'D_type',
                       'header.dialogueInfo.topic': 'D_topic',
                       'header.dialogueInfo.dialogueID':'D_id',
                       'utteranceID':'U_id',
                       'participantID':'P_id',
                       'turnID':'T_id',
                       'date':'U_date',
                       'gender':'P_gender',
                       'residentialProvince':'P_residential',
                       'age':'P_age'}, inplace = True)
  # 대화 id, utterance id 를 index로 함. 마찬가지로 수정해도 됨.
  df.set_index(['D_id','U_id'], inplace=True)
  return df


In [2]:
# 행사.json, 식음료.json 등 여러 데이터셋을 하나로 만들기 위한 함수
def concat_json2df (file_list):
  result = None

  for i, file in enumerate(file_list):
    df = json2df(file)
    if (i==0):
      result = df.copy()
    else:
      result = pd.concat([result,df])
  return result

In [9]:
f_list = ['/content/gdrive/MyDrive/cose461/행사.json',
          '/content/gdrive/MyDrive/cose461/식음료.json']

test3 = concat_json2df(f_list)

test3.describe()

,utterance,P_id,U_date,T_id,time,P_num,U_num,T_num,D_type,D_topic,P_age,P_residential,P_gender
count,3381752,3381752,3381752,3381752,3381752,3381752,3381752,3381752,3381752,3381752,3381752,3381752,3381752
unique,2543121,9,2873,20,1440,8,23,17,2,2,9,18,2
top,#@시스템#사진#,P01,2020-12-21,T3,17:55:00,2,20,8,일상 대화,식음료,20대,서울특별시,여성
freq,38365,1759889,8714,455462,5287,2981713,499120,599571,3380905,1821031,2470129,969946,2713080


In [216]:
# example of json2df function
x = json2df('/content/gdrive/MyDrive/cose461/행사3.json')
x

utterance  ... P_gender
D_id                                 U_id                               ...         
223740cf-606e-5149-820c-7dce69513da1 U1                      8월 말도 ㄱㅊ해  ...       여성
                                     U2                   그때도 글ㅋㅔ 안추울걸  ...       여성
                                     U3                       29일로 할까?  ...       여성
                                     U4     날짜라도 일단 확정하면 ㅋㅋㅋ 찾기 수월하니까~  ...       여성
                                     U5                   응응 그럼 29일로가자  ...       여성
                                     U6    웅웅ㅋㅋ #@이름#이만 되면 날짜는 결정~!~~!  ...       여성
                                     U7                             웅웅  ...       여성
                                     U8                         이제봐땨ㅎㅎ  ...       여성
                                     U9                8월 말은 괜찮을거가태ㅎㅎ!  ...       여성
                                     U10                       #@이모티콘#  ...       여성
d7216603-6193-55a9-b6b8-58a7b6fdb433 U1                        허니가 나중에  ...       남성
                                     U2                        알려줄게용헤헤  ...       남성
                                     U3                           웅웅헤헤  ...       여성
                                     U4                          허니 이제  ...       남성
                                     U5                    고대랑 먼지 볼려구우  ...       여성
                                     U6                           집가려구  ...       남성
                                     U7                     엌ㅋㅋㅋㅋㅋㅋㅋㅋㅋ  ...       남성
                                     U8                     바로 집 와...?  ...       여성
                                     U9                            집오면  ...       남성
                                     U10                       점심먹고 가면  ...       남성
                                     U11                    6시쯤되지않을까..  ...       남성
                                     U12                           그치흑  ...       여성

[22 rows x 13 columns]

In [214]:
# example of json2df function
test2 = json2df('/content/gdrive/MyDrive/cose461/행사.json')


In [217]:
test2.describe()

,utterance,P_id,U_date,T_id,time,P_num,U_num,T_num,D_type,D_topic,P_age,P_residential,P_gender
count,1560721,1560721,1560721,1560721,1560721,1560721,1560721,1560721,1560721,1560721,1560721,1560721,1560721
unique,1198121,9,2710,20,1440,8,23,17,2,1,8,18,2
top,#@이모티콘#,P01,2020-07-20,T2,17:53:00,2,20,8,일상 대화,행사,20대,서울특별시,여성
freq,15155,805283,2946,204808,2408,1349965,257640,256863,1560559,1560721,1146489,462031,1232669


In [163]:
meta_list = [['header', 'dialogueInfo', 'numberOfParticipants'],
             ['header', 'dialogueInfo', 'numberOfUtterances'],
             ['header', 'dialogueInfo', 'numberOfTurns'],
             ['header', 'dialogueInfo', 'type'],
             ['header', 'dialogueInfo', 'topic'],
             ['header', 'dialogueInfo', 'dialogueID'],
             
            ]
             #['header', 'participantsInfo', 'residentialProvince'],
             #['header', 'participantsInfo', 'gender'],
             #['header', 'participantsInfo', 'participantID']]
df1 = pd.json_normalize(jdata['data'][:], record_path=['body'], meta= meta_list, errors='ignore')
df2 = pd.json_normalize(jdata['data'], record_path=['header', 'participantsInfo'],  meta= [['header', 'dialogueInfo', 'dialogueID']], errors='ignore')
df2

pd.merge(left = df1 , right = df2, how = "left", on = ["header.dialogueInfo.dialogueID", 'participantID' ],sort=False)


,utterance,utteranceID,participantID,date,turnID,time,header.dialogueInfo.numberOfParticipants,header.dialogueInfo.numberOfUtterances,header.dialogueInfo.numberOfTurns,header.dialogueInfo.type,header.dialogueInfo.topic,header.dialogueInfo.dialogueID,age,residentialProvince,gender
0,8월 말도 ㄱㅊ해,U1,P01,2020-07-01,T1,11:22:00,3,10,6,일상 대화,행사,223740cf-606e-5149-820c-7dce69513da1,30대,서울특별시,여성
1,그때도 글ㅋㅔ 안추울걸,U2,P01,2020-07-01,T1,11:22:00,3,10,6,일상 대화,행사,223740cf-606e-5149-820c-7dce69513da1,30대,서울특별시,여성
2,29일로 할까?,U3,P01,2020-07-01,T1,11:22:00,3,10,6,일상 대화,행사,223740cf-606e-5149-820c-7dce69513da1,30대,서울특별시,여성
3,날짜라도 일단 확정하면 ㅋㅋㅋ 찾기 수월하니까~,U4,P01,2020-07-01,T1,11:22:00,3,10,6,일상 대화,행사,223740cf-606e-5149-820c-7dce69513da1,30대,서울특별시,여성
4,응응 그럼 29일로가자,U5,P02,2020-07-01,T2,11:22:00,3,10,6,일상 대화,행사,223740cf-606e-5149-820c-7dce69513da1,30대,경기도,여성
5,웅웅ㅋㅋ #@이름#이만 되면 날짜는 결정~!~~!,U6,P01,2020-07-01,T3,11:23:00,3,10,6,일상 대화,행사,223740cf-606e-5149-820c-7dce69513da1,30대,서울특별시,여성
6,웅웅,U7,P02,2020-07-01,T4,11:25:00,3,10,6,일상 대화,행사,223740cf-606e-5149-820c-7dce69513da1,30대,경기도,여성
7,이제봐땨ㅎㅎ,U8,P03,2020-07-01,T5,11:28:00,3,10,6,일상 대화,행사,223740cf-606e-5149-820c-7dce69513da1,30대,서울특별시,여성
8,8월 말은 괜찮을거가태ㅎㅎ!,U9,P03,2020-07-01,T5,11:28:00,3,10,6,일상 대화,행사,223740cf-606e-5149-820c-7dce69513da1,30대,서울특별시,여성
9,#@이모티콘#,U10,P01,2020-07-01,T6,11:29:00,3,10,6,일상 대화,행사,223740cf-606e-5149-820c-7dce69513da1,30대,서울특별시,여성


In [ ]:
#test test test